In [1]:
import zipfile
import io

import sqlite3

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sentence_transformers import SentenceTransformer

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


import numpy as np
import pandas as pd

## **Step 1 - Reading the Tables from Database file**

In [2]:
# Read the code below and write your observation in the next cell

conn = sqlite3.connect('eng_subtitles_database.db')
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())

[('zipfiles',)]


**In the above cell, I am able to read the table inside the database. As mentioned earlier, table name is `zipfiles`. We also know from README.txt that this table contains three columns: 'num', 'name' and 'content'.**

## **Step 2 - Reading the columns of Table**

In [3]:
cursor.execute("PRAGMA table_info('zipfiles')")
cols = cursor.fetchall()
for col in cols:
    print(col[1])

num
name
content


**The above code helps in checking the column names in the database table.**

**Let's now use `SELECT * FROM zipfiles` to read all the data into a `df` variable.**

## **Step 3 - Loading the Database Table inside a Pandas DataFrame**

In [4]:
df = pd.read_sql_query("""SELECT * FROM zipfiles""", conn)
df = df.sample(frac=0.1)
df.head()

,num,name,content
18230,9254799,charmed.s05.e19.nymphs.just.wanna.have.fun.(20...,b'PK\x03\x04\x14\x00\x00\x00\x08\x004\xab\x99V...
5064,9203148,american.gothic.s01.e05.the.artist.in.his.muse...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00[\x8d\x99V...
51601,9397935,call.the.midwife.christmas.special.2022.(2022)...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00%%\x9aVx\x...
11270,9228156,dynasty.s05.e20.first.kidnapping.and.now.theft...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xc8\x94\x...
34937,9321247,harry.wild.s01.e04.an.unhappy.happy.is.a.dange...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x81\xb2\x...


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8250 entries, 18230 to 68553
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   num      8250 non-null   int64 
 1   name     8250 non-null   object
 2   content  8250 non-null   object
dtypes: int64(1), object(2)
memory usage: 257.8+ KB


**Looks like the `content` column donot contain the subtitles text. Instead as mentioned in README.txt, it might be latin-1 encoded.**

## **Step 4 - Printing `content` of 0th Row**

In [6]:
b_data = df.iloc[0, 2]

# here 2 represent the index of content column
# 0 represents the row number

In [7]:
print(b_data)

b'PK\x03\x04\x14\x00\x00\x00\x08\x004\xab\x99V\xae\x9a\x1a+]V\x00\x00\x1f\xdf\x00\x00A\x00\x00\x00Charmed S05E19 Nymphs Just Wanna Have Fun.DVDRip.HI.cc.en.CBS.srt}\xbd\xcd\x92\xdc\xd6\x92?\xb6g\x04\xdf\x01W\x0bkSl\x038\xf8TL\\\x06%Qd\x8f)\x92AR\x97C+f\x81\xaeBw\xe1\xb2\xaaPS\xa8b\xab\xe7M\xbc\xf3\xd6~\x05/\xbc\xf8\xbf\x89_\xc0\xaf\xe0\xfce\xe6I$ZW\x8e\xb83!\xa2\xeb\xe4\x01\xce\xc9\xef\xcf\xff\xf7\xff\xfa\xbf\xb3\xa7O\xd2\xf4\x07\xfc/[5\xa1L\x9e=\xfb{\xa2\x0f\x8aU^\xb5O\x9f\xfc\xfe\xfe\xc5\xdb\xf7\xd7\xef_&\xef\xdf\xbc\xf8r\xfd\xf6\xd5\x7f>}\xf2\xf4I\x1e\x97\xe5\xe5*\xcf\xfd\xb2\xbcYUm\xf1\xf4\xc9\x7f\xbc\xf8\xc7\xf5\xcb\x0f?$\xd7\xc9\xdd\xa5\x9f\xa6d\xff\x90\x0c\x87o\xc3\xb9;\x0f\xe3\xe1\xe9\x93\xfde:\x7f\xff\xadO\xee\xc6\xf3\xb9?$\xbbq:\xd3\xdf\x93\xf3\xb6O\xf6\xdd\xb0\xbb\xc2&\xc16iVu[\xb9MB\xbe\xaaB\xfa\xf4\xc93\x02\x7f\x1ew\x9b\xe4a\xbc$\xd7\xdfo\x92\xdb\xe1 \xff\xe8\xee\xba\xe1\xb0z\xfad\xea\xce\x0f\xa7+\xfc\xf2\xd3\xb6?\xf5\xdfO\xc9a<o\x87\xc3]r;\x9e\xf8\x97x\xcc\xdb\x15q;\x82^\

**From the content, it appears to start with the bytes "PK\x03\......", which suggests that it might be a ZIP archive file. How do I know it? Experience! I have worked with something similar earlier.**

## **Step 5 - Unzipping the content of 380th row and decoding using `latin-1`**

In [8]:
import zipfile
import io

# Assuming 'content' is the binary data from your database
binary_data = df.iloc[380, 2]

# Decompress the binary data using the zipfile module
with io.BytesIO(binary_data) as f:
    with zipfile.ZipFile(f, 'r') as zip_file:
        # Reading only one file in the ZIP archive
        subtitle_content = zip_file.read(zip_file.namelist()[0])

# Now 'subtitle_content' should contain the extracted subtitle content
print(subtitle_content.decode('latin-1'))  # Assuming the content is latin-1 encoded text

ï»¿[Script Info]
Title: Default file
ScriptType: v4.00+
WrapStyle: 0
PlayResX: 720
PlayResY: 480
ScaledBorderAndShadow: yes
Audio File: 
Video File: 
Video Aspect Ratio: 0
Video Zoom: 6
Video Position: 0

[V4+ Styles]
Format: Name, Fontname, Fontsize, PrimaryColour, SecondaryColour, OutlineColour, BackColour, Bold, Italic, Underline, StrikeOut, ScaleX, ScaleY, Spacing, Angle, BorderStyle, Outline, Shadow, Alignment, MarginL, MarginR, MarginV, Encoding
Style: Dialogue1,Tahoma,32,&H00BABABA,&H000000FF,&H1F000000,&HC7000000,0,0,0,0,100,100,0,0,1,1.4,1.7,2,60,60,15,1
Style: Dialogue2,Tahoma,29,&H00BABABA,&H000000FF,&H1F000000,&HC7000000,0,0,0,0,100,100,0,0,1,1.4,1.7,2,60,60,15,1
Style: Dialogue3,Tahoma,25,&H00BABABA,&H000000FF,&H1F000000,&HC7000000,0,0,0,0,100,100,0,0,1,1.4,1.7,2,60,60,15,1

[Events]
Format: Layer, Start, End, Style, Name, MarginL, MarginR, MarginV, Effect, Text
Dialogue: 0,0:00:05.23,0:00:06.67,Dialogue2,Unknown,0000,0000,0000,,{\an4\pos(254,368)}Mom, you're going
Dialogu

## **Step 6 - Applying the above Function on the Entire Data**

In [9]:
count = 0

def decode_method(binary_data):
    global count
    # Decompress the binary data using the zipfile module
    # print(count, end=" ")
    count += 1
    with io.BytesIO(binary_data) as f:
        with zipfile.ZipFile(f, 'r') as zip_file:
            # Assuming there's only one file in the ZIP archive
            subtitle_content = zip_file.read(zip_file.namelist()[0])
    
    # Now 'subtitle_content' should contain the extracted subtitle content
    return subtitle_content.decode('latin-1')  # Assuming the content is UTF-8 encoded text

In [10]:
df['file_content'] = df['content'].apply(decode_method)

df.head()

,num,name,content,file_content
18230,9254799,charmed.s05.e19.nymphs.just.wanna.have.fun.(20...,b'PK\x03\x04\x14\x00\x00\x00\x08\x004\xab\x99V...,"ï»¿1\r\n00:00:01,835 --> 00:00:04,269\r\n[PANP..."
5064,9203148,american.gothic.s01.e05.the.artist.in.his.muse...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00[\x8d\x99V...,"ï»¿1\r\n00:00:02,102 --> 00:00:05,095\r\n<i>Pr..."
51601,9397935,call.the.midwife.christmas.special.2022.(2022)...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00%%\x9aVx\x...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nAdver..."
11270,9228156,dynasty.s05.e20.first.kidnapping.and.now.theft...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xc8\x94\x...,"ï»¿1\r\n00:00:04,968 --> 00:00:06,922\r\nBLAKE..."
34937,9321247,harry.wild.s01.e04.an.unhappy.happy.is.a.dange...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x81\xb2\x...,"1\r\n00:00:06,000 --> 00:00:12,074\r\nWatch an..."


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8250 entries, 18230 to 68553
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   num           8250 non-null   int64 
 1   name          8250 non-null   object
 2   content       8250 non-null   object
 3   file_content  8250 non-null   object
dtypes: int64(1), object(3)
memory usage: 322.3+ KB


In [12]:
df.tail()

,num,name,content,file_content
24646,9279299,in.sickness.and.in.health.s05.e05.episode.5.5....,b'PK\x03\x04\x14\x00\x00\x00\x08\x00s\xa1\x99V...,"ï»¿1\r\n00:00:02,800 --> 00:00:05,758\r\n(# CH..."
39802,9344377,too.hot.to.handle.s04.e04.flavia.of.the.month....,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xf0\xbc\x...,"ï»¿1\r\n00:00:06,000 --> 00:00:12,074\r\nAdver..."
66441,9455317,south.park.s26.e03.japanese.toilets.(2023).eng...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00A_\x9aV\xf...,"ï»¿1\r\n00:00:40,123 --> 00:00:41,373\r\nSee, ..."
74986,9490207,everybody.hates.chris.s04.e08.everybody.hates....,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x11p\x9aV...,"1\r\n00:00:03,671 --> 00:00:05,939\r\nGot a li..."
68553,9463438,legend.of.the.seeker.s01.e17.deception.(2009)....,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xbbd\x9aV...,"ï»¿1\r\n00:00:03,004 --> 00:00:04,448\r\nYou b..."


# Data Cleaning

In [13]:
import re

def clean_text(text):
    cleaned_text = re.sub(r'[^a-zA-Z\s]', '', text)
    cleaned_text = re.sub(r'[^\x00-\x7F]+', '', cleaned_text)
    cleaned_text = re.sub(r'\d{2}:\d{2}:\d{2}\d{3} --> \d{2}:\d{2}:\d{2},\d{3}', '', cleaned_text)
    cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
    cleaned_text = cleaned_text.lower().strip()
    
    return cleaned_text

In [14]:
df['file_content'] = df['file_content'].apply(clean_text)

In [15]:
df

,num,name,content,file_content
18230,9254799,charmed.s05.e19.nymphs.just.wanna.have.fun.(20...,b'PK\x03\x04\x14\x00\x00\x00\x08\x004\xab\x99V...,panpipe playing xavier i guess my invitation m...
5064,9203148,american.gothic.s01.e05.the.artist.in.his.muse...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00[\x8d\x99V...,ipreviously oni american gothic you left this ...
51601,9397935,call.the.midwife.christmas.special.2022.(2022)...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00%%\x9aVx\x...,advertise your product or brand here contact w...
11270,9228156,dynasty.s05.e20.first.kidnapping.and.now.theft...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xc8\x94\x...,blake well theres no proof that heart arrhythm...
34937,9321247,harry.wild.s01.e04.an.unhappy.happy.is.a.dange...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x81\xb2\x...,watch any video online with opensubtitles free...
...,...,...,...,...
24646,9279299,in.sickness.and.in.health.s05.e05.episode.5.5....,b'PK\x03\x04\x14\x00\x00\x00\x08\x00s\xa1\x99V...,chas and dave in sickness and in health now my...
39802,9344377,too.hot.to.handle.s04.e04.flavia.of.the.month....,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xf0\xbc\x...,advertise your product or brand here contact w...
66441,9455317,south.park.s26.e03.japanese.toilets.(2023).eng...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00A_\x9aV\xf...,see stan you gotta choose if you wanna be in g...
74986,9490207,everybody.hates.chris.s04.e08.everybody.hates....,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x11p\x9aV...,got a little something for you man she is a fl...


## CHUNKING DOCUMENTS !

In [16]:
def chunk_text(text, chunk_size=500, overlap=50):
    chunks = []
    words = text.split()
    for start in range(0, len(words), chunk_size - overlap):
        chunks.append(' '.join(words[start:start + chunk_size]))
    return chunks

In [17]:
df['chunked_text'] = df['file_content'].apply(lambda x: chunk_text(x))

In [20]:
df

,num,name,content,file_content,chunked_text
18230,9254799,charmed.s05.e19.nymphs.just.wanna.have.fun.(20...,b'PK\x03\x04\x14\x00\x00\x00\x08\x004\xab\x99V...,panpipe playing xavier i guess my invitation m...,[panpipe playing xavier i guess my invitation ...
5064,9203148,american.gothic.s01.e05.the.artist.in.his.muse...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00[\x8d\x99V...,ipreviously oni american gothic you left this ...,[ipreviously oni american gothic you left this...
51601,9397935,call.the.midwife.christmas.special.2022.(2022)...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00%%\x9aVx\x...,advertise your product or brand here contact w...,[advertise your product or brand here contact ...
11270,9228156,dynasty.s05.e20.first.kidnapping.and.now.theft...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xc8\x94\x...,blake well theres no proof that heart arrhythm...,[blake well theres no proof that heart arrhyth...
34937,9321247,harry.wild.s01.e04.an.unhappy.happy.is.a.dange...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x81\xb2\x...,watch any video online with opensubtitles free...,[watch any video online with opensubtitles fre...
...,...,...,...,...,...
24646,9279299,in.sickness.and.in.health.s05.e05.episode.5.5....,b'PK\x03\x04\x14\x00\x00\x00\x08\x00s\xa1\x99V...,chas and dave in sickness and in health now my...,[chas and dave in sickness and in health now m...
39802,9344377,too.hot.to.handle.s04.e04.flavia.of.the.month....,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xf0\xbc\x...,advertise your product or brand here contact w...,[advertise your product or brand here contact ...
66441,9455317,south.park.s26.e03.japanese.toilets.(2023).eng...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00A_\x9aV\xf...,see stan you gotta choose if you wanna be in g...,[see stan you gotta choose if you wanna be in ...
74986,9490207,everybody.hates.chris.s04.e08.everybody.hates....,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x11p\x9aV...,got a little something for you man she is a fl...,[got a little something for you man she is a f...


In [18]:
df['chunked_text']

18230    [panpipe playing xavier i guess my invitation ...
5064     [ipreviously oni american gothic you left this...
51601    [advertise your product or brand here contact ...
11270    [blake well theres no proof that heart arrhyth...
34937    [watch any video online with opensubtitles fre...
                               ...                        
24646    [chas and dave in sickness and in health now m...
39802    [advertise your product or brand here contact ...
66441    [see stan you gotta choose if you wanna be in ...
74986    [got a little something for you man she is a f...
68553    [you be the hiders ill be the seeker all right...
Name: chunked_text, Length: 8250, dtype: object

## BERT EMBEDDINGS for chunked data

In [19]:
## BERT - “SentenceTransformers” : Generates embeddings which encode semantic information.
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [21]:
df["chunk_embeddings"] = df["chunked_text"].apply(lambda x: model.encode(x))

In [22]:
df['chunk_embeddings']

18230    [[0.035840474, 0.031651206, -0.0029583517, 0.0...
5064     [[0.021123009, 0.07433403, 0.0057303784, 0.003...
51601    [[0.05199184, 0.048980944, -0.020183316, 0.051...
11270    [[0.023384012, 0.118389376, -0.004994454, -0.0...
34937    [[0.016029937, 0.022402475, 0.016657071, 0.011...
                               ...                        
24646    [[0.023927758, 0.02417276, 0.0030255115, -0.04...
39802    [[0.08386179, 0.044809118, -0.005196778, -0.00...
66441    [[0.0523822, 0.080413654, 0.0035326094, 0.0235...
74986    [[0.040766783, 0.07189492, -0.013499531, 0.003...
68553    [[0.01893767, 0.07143243, 0.012997342, 0.04420...
Name: chunk_embeddings, Length: 8250, dtype: object

In [23]:
df.to_csv('embedded_df_10_percent.csv', index=False)
# Download link
from IPython.display import FileLink
FileLink('embedded_df_10_percent.csv')

/Users/eleshalapravalika/Downloads/SEARCH ENGINE PROJECT/embedded_df_10_percent.csv

In [24]:
df_embed = pd.read_csv('embedded_df_10_percent.csv')

In [25]:
df_embed

,num,name,content,file_content,chunked_text,chunk_embeddings
0,9254799,charmed.s05.e19.nymphs.just.wanna.have.fun.(20...,b'PK\x03\x04\x14\x00\x00\x00\x08\x004\xab\x99V...,panpipe playing xavier i guess my invitation m...,['panpipe playing xavier i guess my invitation...,[[ 0.03584047 0.03165121 -0.00295835 ... 0.0...
1,9203148,american.gothic.s01.e05.the.artist.in.his.muse...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00[\x8d\x99V...,ipreviously oni american gothic you left this ...,['ipreviously oni american gothic you left thi...,[[ 0.02112301 0.07433403 0.00573038 ... 0.0...
2,9397935,call.the.midwife.christmas.special.2022.(2022)...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00%%\x9aVx\x...,advertise your product or brand here contact w...,['advertise your product or brand here contact...,[[ 0.05199184 0.04898094 -0.02018332 ... 0.0...
3,9228156,dynasty.s05.e20.first.kidnapping.and.now.theft...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xc8\x94\x...,blake well theres no proof that heart arrhythm...,['blake well theres no proof that heart arrhyt...,[[ 0.02338401 0.11838938 -0.00499445 ... 0.0...
4,9321247,harry.wild.s01.e04.an.unhappy.happy.is.a.dange...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x81\xb2\x...,watch any video online with opensubtitles free...,['watch any video online with opensubtitles fr...,[[ 0.01602994 0.02240247 0.01665707 ... 0.0...
...,...,...,...,...,...,...
8245,9279299,in.sickness.and.in.health.s05.e05.episode.5.5....,b'PK\x03\x04\x14\x00\x00\x00\x08\x00s\xa1\x99V...,chas and dave in sickness and in health now my...,['chas and dave in sickness and in health now ...,[[ 2.3927758e-02 2.4172761e-02 3.0255115e-03...
8246,9344377,too.hot.to.handle.s04.e04.flavia.of.the.month....,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\xf0\xbc\x...,advertise your product or brand here contact w...,['advertise your product or brand here contact...,[[ 0.08386179 0.04480912 -0.00519678 ... 0.0...
8247,9455317,south.park.s26.e03.japanese.toilets.(2023).eng...,b'PK\x03\x04\x14\x00\x00\x00\x08\x00A_\x9aV\xf...,see stan you gotta choose if you wanna be in g...,['see stan you gotta choose if you wanna be in...,[[ 0.0523822 0.08041365 0.00353261 ... 0.0...
8248,9490207,everybody.hates.chris.s04.e08.everybody.hates....,b'PK\x03\x04\x14\x00\x00\x00\x08\x00\x11p\x9aV...,got a little something for you man she is a fl...,['got a little something for you man she is a ...,[[ 0.04076678 0.07189492 -0.01349953 ... 0.0...
